<a href="https://colab.research.google.com/github/ucfilho/RTC_case_02_optimization_rest/blob/master/RTC_DE_case_02_aproach_Fu_abril_20_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/ucfilho/raianars_RTC_optimization #clonar do Github
%cd raianars_RTC_optimization
import de_bounds_matriz

fatal: destination path 'raianars_RTC_optimization' already exists and is not an empty directory.
/content/raianars_RTC_optimization


In [0]:
# keep the split stream in last position ?????

In [0]:
def SharkBuildMatriz(x):
  # Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha=SharkBuildMatriz(x)
  # add one position in three variables (Ind_J,Ind_I,deltaT)
  # Alfa and Split (split choose the moment and alpha the fraction)
  # Split=which stream
  # Alfa=fraction 
  
  global Trocas
  NPAR=1
  #cols=len(tempInicial)
  size=int(3*Trocas+3) # number 3 represent split, alfa and WhenSplit
  # add one exchange
  x=np.array(x).reshape(1,size)
  # print('x=');print(x)
  Ind_I=np.zeros((NPAR,Trocas)).astype(np.int64)
  Ind_J=np.zeros((NPAR,Trocas)).astype(np.int64)
  
  Tout=np.zeros((NPAR,Trocas))
  deltaT=np.zeros((NPAR,Trocas))
  for i in range(NPAR):
    Alpha=x[i,size-3]
    Split=round(x[i,size-2]).astype(np.int64)
    WhenSplit=round(x[i,size-1]).astype(np.int64)
    for j in range(Trocas):
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas] 

  return Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha
  #Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha=SharkBuildMatriz(x)

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  
  NPAR=1
  '''
  funr=np.zeros((NPAR))
  cols=len(mCp)
  Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  Tout, Erro=Temperatura(x)
  areaTrocadores =Area(x)
  areaUtilidade, cargaUQ,cargaUF = Utilidades(tempAlvo,Tout,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr=cost

  for k in range(NPAR):
    if(Erro[k]>0):
      funr[k]=1e99
  funr = np.where(np.isnan(cost), 1e99, cost)
  '''
  funr=1e99 # fake line (temporary)
  return funr 

In [0]:
def ChangeTemp(ki,kj,i,j,Tout,Cp,deltaT):
  global NPAR
  erro=np.zeros((NPAR)) # erro=1 if there  is an error : penalties
  tempSaidaQ=Tout[ki,i]+deltaT[ki,kj]
  tempSaidaF=-Cp[kj]*deltaT[ki,kj]/(Cp[j])+Tout[ki,j]
  flag=0 # flag : logical test
  if(Tout[ki,i] > Tout[ki,j]): #Tquente maior Tfria: vaores atuais
    if(tempSaidaQ > tempSaidaF):#Tquente maior Tfria: saidas
      flag=1
  if(flag>0):
    Tout[ki,i]=tempSaidaQ
    Tout[ki,j]=tempSaidaF
  else:
    erro[ki]=1

  Thot=np.copy(Tout[ki,i]) # value used to return original value (if request)
  Tcold=np.copy(Tout[ki,j]) # value used to return original value (if request)
  
  return Tout,erro,Thot,Tcold
  # Tout,erro,Thot,Tcold=ChangeTemp(ki,kj,i,j,Tout,Cp,deltaT)       
 

In [0]:
def Utility(Trocas,Rec_Temp,tempAlvo):
  df=pd.DataFrame(Rec_Temp)
  tempFinal=np.zeros((Trocas-1))
  row=len(df.iloc[:,0])
  col=len(df.iloc[0,:])-1
  for i in range(row):
    for j in range(col):
      tempFinal[j]=np.copy(df.iloc[i,j]) 

In [0]:
def Temperatura(x):
  global Trocas, mCp,tempInicial,tempAlvo,Ttest

  NPAR=1
  Tin=np.zeros(Trocas)
  Tout=np.zeros((NPAR,Trocas))
  mCpin=np.zeros(Trocas)
  
  rows=Trocas 
  cols=Trocas
  Rec_Temp=np.zeros((rows+4,cols))# four extra positions to better view and one to split

  Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha=SharkBuildMatriz(x)


  if(WhenSplit==Trocas-1):
    Ind_1st=WhenSplit-2
    Ind_2nd=Trocas-1
  else:
    Ind_1st=WhenSplit
    Ind_2nd=Trocas-1
  
  #=====================
  # part B: allocating split in cold or hot stream
  # Hot=1 means hot stream is split
  # Hot=0 means hot stream is not split (so cold stream is split)
  if(tempInicial[Split]>tempAlvo[Split]): # hot stream is spliting
    Hot=1 
    Ind_I[0,Ind_1st]=Split # two streams are correct by split information
    Ind_I[0,Ind_2nd]=Trocas-1 # two streams are correct by split information
  else: # cold stream is spliting
    Hot=0 
    Ind_J[0,Ind_1st]=Split # two streams are correct by split information
    Ind_J[0,Ind_2nd]=Trocas-1 # two streams are correct by split information

  #=====================
  # part C: bulding mCp with split values and temperatures with split values
  for kk in range(Trocas-1):  # ???(Trocas-1) should be replaced by (Trocas-2)?
    Tin[kk]=np.copy(tempInicial[kk])
    mCpin[kk]=mCp[kk]

  Tin[Trocas-1]=tempInicial[Split]
  mCpin[Trocas-1]=mCpin[Split]    


  mCpout=np.copy(mCpin)
  if(Split==Trocas-1):
    mCpout[Split-1]= Alpha*mCpin[Split]
    mCpout[Split]= (1-Alpha)*mCpin[Split]
  else:
    mCpout[Split]=Alpha*mCpin[Split]
    mCpout[Trocas-1]= (1-Alpha)*mCpin[Split]
  

  for ki in range(NPAR):
    for kj in range(Trocas): 
      Tout[ki,kj]=np.copy(Tin[kj])

  # Lists Rec_Temp is used to build graph 
  for i in range(2):
    for j in range(cols):
      Rec_Temp[i,j]=np.copy(Tout[0,j])

  Ttest=np.copy(Tout)
  Grab=0 #grab is evaluation: 0: no split, 1:start slit, 2:conclude split 
  #=====================
  # part D: start get temperatures using informations build (A,B,C)
  Cont=1
  for ki in range(NPAR):
    for kj in range(Trocas):
      Cont=Cont+1
      k=kj
      if(kj==WhenSplit):
        Cp=np.copy(mCpout)
        grab=1
      elif(kj==WhenSplit+1):
        Cp=np.copy(mCpout)
        grab=2
      else:
        Cp=np.copy(mCpin)
        grab=0
      
      if(grab==2):
        if(Hot==1):
          i=Trocas-1
        else:
          j=Trocas-1
      else:
        i=Ind_I[ki,kj]
        j=Ind_J[ki,kj]

      
      Tout,erro,Thot,Tcold=ChangeTemp(ki,kj,i,j,Tout,Cp,deltaT) 

      for j in range(Trocas):
        Rec_Temp[Cont,j]=np.copy(Tout[0,j])

      
      if(grab==2):
        Taux=Alpha*Tout[ki,Split]+(1-Alpha)*Tout[ki,Trocas-1]
        Tout[ki,Trocas-1]=np.copy(Taux)
        Tout[ki,Split]=np.copy(Taux)
        Cont=Cont+1
        for j in range(Trocas):
          Rec_Temp[Cont,j]=np.copy(Tout[0,j])
        Cont=Cont+1

      if(grab == 0):
        if(Hot==1):
          Tout[ki,Trocas-1]=Tout[ki,Split]
        else:
          Tout[ki,Trocas-1]=Tout[ki,Split]

      for j in range(cols):
        Rec_Temp[Cont,j]=np.copy(Tout[0,j])


    
  return Tout,erro,Rec_Temp

  # return Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha # fake return until test
  # return Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha,Tin # fake return until test
  # return Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha,Tin, mCpin # fake return until test
  # return Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha,Tin, mCpin, mCpout # fake return until test

In [0]:
#********************PROGRAMA PRINCIPAL*************************

Trocas=5
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453.0,393.0,523.0,533.0] #[K]
tempAtual=np.copy(tempInicial) 
tempAlvo=[513.0,508.0,403.0,433.0] #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

# the last position in MAX and MIN vector represent which moment the split 
#    occur not which stream (split represent which stream) 
#    for this reason it reach larger value
#MAX=np.array([3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,1,3,4]) # ,1,3,4]) Alpha,Split,WhenSplit
#MIN=np.array([0,0,0,0,0,0,0,0,0,0,-100,-100,-100,-100,-100,0,0,0])

MAX=np.array([3,3,3,3,3,1,1,1,1,1,0,0,0,0,0,1,3,4]) # ,1,3,4]) Alpha,Split,WhenSplit
MIN=np.array([2,2,2,2,2,0,0,0,0,0,-20,-20,-20,-20,-20,0,0,0])
NPAR=1
cols=int(3*Trocas+3)
x=np.zeros((NPAR,cols))

for i in range(NPAR):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]





bounds=[]
for k in range(len(MAX)):
  bounds.append((MIN[k],MAX[k]))


#print(bounds)



In [9]:

Ttest=np.zeros((1,Trocas))

Ind_I,Ind_J,deltaT, Split, WhenSplit, Alpha=SharkBuildMatriz(x)
Tout,erro,Rec_Temp=Temperatura(x)

print(Ind_I)
print(Ind_J)
print(deltaT)
print('Split=',Split)
print(Alpha)
print(WhenSplit)
print("#####------######")
print(Ttest)
print(tempInicial)



[[2 2 2 2 2]]
[[0 1 0 0 0]]
[[ -1.48720033  -3.4545016  -16.45350725 -17.09428843  -6.43421172]]
Split= 0
0.5285272582367072
1
#####------######
[[453. 393. 523. 533. 453.]]
[453.0, 393.0, 523.0, 533.0]


In [10]:

print('=====================')
Tout,erro,Rec_Temp =Temperatura(x)
df=pd.DataFrame(Rec_Temp)
print('*****',Tout)
print(erro)
print(df)


***** [[475.20523268 393.         484.51050238 533.         475.20523268]]
[1.]
            0      1           2      3           4
0  453.000000  393.0  523.000000  533.0  453.000000
1  453.000000  393.0  523.000000  533.0  453.000000
2  454.487200  393.0  521.512800  533.0  454.487200
3  457.755245  393.0  518.058298  533.0  454.487200
4  457.755245  393.0  501.604791  533.0  467.573991
5  462.384516  393.0  501.604791  533.0  462.384516
6  462.384516  393.0  501.604791  533.0  462.384516
7  475.205233  393.0  484.510502  533.0  475.205233
8  475.205233  393.0  484.510502  533.0  475.205233


In [11]:
print(WhenSplit)

1


Text(0, 0.5, 'Temperature (K)')

In [12]:
#df = df[:-1]
Imagem=len(df.iloc[:,0]) # Imagens+5 to get better graph postions
Change=range(Imagem)
for k in range(Trocas):
  plt.plot(Change,df.iloc[:,k])

plt.xlabel('Stages')
plt.ylabel('Temperature (K)')

In [13]:
print(df)

            0      1           2      3           4
0  453.000000  393.0  523.000000  533.0  453.000000
1  453.000000  393.0  523.000000  533.0  453.000000
2  454.487200  393.0  521.512800  533.0  454.487200
3  457.755245  393.0  518.058298  533.0  454.487200
4  457.755245  393.0  501.604791  533.0  467.573991
5  462.384516  393.0  501.604791  533.0  462.384516
6  462.384516  393.0  501.604791  533.0  462.384516
7  475.205233  393.0  484.510502  533.0  475.205233
8  475.205233  393.0  484.510502  533.0  475.205233


In [14]:
print(Split)
print(WhenSplit)
print(tempInicial)


0
1
[453.0, 393.0, 523.0, 533.0]


In [16]:
tempFinal=np.zeros((Trocas-1))
row=len(df.iloc[:,0])
col=len(df.iloc[0,:])-1
for i in range(row):
  for j in range(col):
    tempFinal[j]=np.copy(df.iloc[i,j])
print(tempFinal)
print(row)
print(col)

[475.20523268 393.         484.51050238 533.        ]
9
4


In [0]:
'''
fobj=FUN
mut=0.5
crossp=0.5
NPAR=100
its=200
cols=12
x=np.zeros((NPAR,cols))
for i in range(NPAR):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

x,BEST,FOBEST,XY,BEST_XY=de_bounds_matriz.de(bounds, mut, crossp, 
                                             NPAR, its,fobj,x)
'''

In [0]:
#print('FOBEST=');print(FOBEST)

In [0]:
#print(BEST)

In [0]:
'''
Busca=Numb(BEST)
print(pd.DataFrame(Busca))
df=pd.DataFrame(Busca)
'''

In [0]:
print(Alpha)